## 사전준비
### 1. 랭체인 패키지 설치

In [ ]:
%pip install -qU openai langchain langchain-naver 

### 2. 공통 모듈 import

In [ ]:
import os
import getpass
import uuid
import re
from urllib.parse import urlparse
import http
import json
import time

### 3. API 키 발급 받기

In [ ]:
os.environ["CLOVASTUDIO_API_KEY"] = getpass.getpass("CLOVA Studio API Key: ")

## 문서 전처리하기

### 1. PDF 문서에서 텍스트와 이미지 추출하기 (Load)

In [ ]:
%pip install pymupdf

In [ ]:

def confirm(t):
    a = [] 
    a.append(t)
    a.append(t+1)
    return a
a = []
a.extend(confirm(1))
a.extend(confirm(2))
print(a)

In [ ]:
import fitz  # PyMuPDF
from langchain_core.documents import Document

def extract_documents_from_pdf(pdf_path: str, output_dir: str = "data/extracted_images_문서"):
    os.makedirs(output_dir, exist_ok=True)

    merged_text_path = os.path.join(output_dir, "merged_text.txt")
    merged_text = ""

    doc = fitz.open(pdf_path)
    documents = []

    for i, page in enumerate(doc):
        page_number = i + 1
        page_text = page.get_text("text").strip()
        images_info = []

        # 이미지 추출
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_filename = f"page_{page_number}_img_{img_index+1}.{image_ext}"
            image_path = os.path.join(output_dir, image_filename)

            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)

            images_info.append(image_path)

        # LangChain Document로 변환
        documents.append(Document(
            page_content=page_text,
            metadata={
                "source": os.path.basename(pdf_path),
                "page": page_number,
                "images": ", ".join(images_info)
            }
        ))

        # 병합 텍스트 저장용
        merged_text += f"\n\n--- Page {page_number} ---\n\n{page_text}"

    # 전체 텍스트 저장
    with open(merged_text_path, "w", encoding="utf-8") as f:
        f.write(merged_text)

    return documents, merged_text_path

pdf_path = "data/modeltuning.pdf"
docs, merged_path = extract_documents_from_pdf(pdf_path)

print(f"추출된 문서 페이지 수: {len(docs)}")
print(f"병합된 텍스트 경로: {merged_path}")
print(docs[0])  # 하나 확인

In [ ]:
%pip install -qU Pillow

In [ ]:
from PIL import Image
from pathlib import Path
import shutil

def check_and_resize_image_to_outdir(
    path: Path,
    outdir: Path,
    allowed_formats=("PNG", "JPEG", "WEBP", "BMP"),
    max_bytes=20 * 1024 * 1024,
    max_length=2240,
    max_ratio=4.5,
    save_format="PNG"
):
    try:
        # 용량 초과 확인
        if path.stat().st_size > max_bytes:
            print(f"[✘] 용량 초과: {path.name}")
            return

        with Image.open(path) as image:
            format = image.format.upper()
            if format not in allowed_formats:
                print(f"[✘] 포맷 불가: {path.name} ({format})")
                return

            w, h = image.size
            ratio = max(w, h) / min(w, h)
            needs_resize = max(w, h) > max_length or ratio > max_ratio

            if not needs_resize:
                # 조건 만족 → 그대로 복사
                dest = outdir / path.name
                shutil.copy(path, dest)
                print(f"[✓] 조건 만족 → 복사됨: {path.name}")
                return

            # 리사이즈 크기 계산
            if ratio > max_ratio:
                if w > h:
                    new_w = min(w, max_length)
                    new_h = int(new_w / max_ratio)
                else:
                    new_h = min(h, max_length)
                    new_w = int(new_h / max_ratio)
            else:
                if w >= h:
                    new_w = min(w, max_length)
                    new_h = int(h * (new_w / w))
                else:
                    new_h = min(h, max_length)
                    new_w = int(w * (new_h / h))

            resized = image.resize((new_w, new_h), Image.LANCZOS).convert("RGB")
            dest = outdir / path.name
            resized.save(dest, format=save_format, optimize=True)
            print(f"[✔] 리사이즈됨 → 저장됨: {dest.name} ({new_w}x{new_h})")

    except Exception as e:
        print(f"[✘] 처리 실패: {path.name} → {e}")


In [ ]:
from pathlib import Path

input_dir = Path("data/extracted_images_문서")
output_dir = Path("data/filtered_images")
output_dir.mkdir(parents=True, exist_ok=True)

valid_exts = [".png", ".jpg", ".jpeg", ".webp", ".bmp"]
image_files = [p for p in input_dir.glob("*") if p.suffix.lower() in valid_exts]

print(f"총 {len(image_files)}개의 이미지 처리 시작")

for img_path in image_files:
    check_and_resize_image_to_outdir(img_path, outdir=output_dir)


In [ ]:
# 네이버 클라우드에서 발급받은 키를 입력하세요
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("NCP Access Key: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("NCP Secret Key: ")

# 기본 리전 설정
os.environ["AWS_DEFAULT_REGION"] = "kr"

## Ncloud Object Storage 에 이미지 저장 및 링크 생성

In [ ]:
%pip install boto3

In [ ]:
from glob import glob
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import mimetypes

# 설정
BUCKET_NAME = "multi-rag-techseminar"
LOCAL_FOLDER = "data/filtered_images"
ENDPOINT_URL = "https://kr.ncloudstorage.com"
REGION = os.environ["AWS_DEFAULT_REGION"]

ACCESS_KEY = os.environ["AWS_ACCESS_KEY_ID"]
SECRET_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]

# boto3 클라이언트 초기화
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
    config=Config(signature_version="s3v4")
)

# 1. 버킷 생성
try:
    s3.head_bucket(Bucket=BUCKET_NAME)
    print(f"이미 존재하는 버킷입니다: {BUCKET_NAME}")
except ClientError as e:
    if e.response['Error']['Code'] == '404':
        print(f"버킷이 존재하지 않아 생성합니다: {BUCKET_NAME}")
        s3.create_bucket(Bucket=BUCKET_NAME)
    else:
        raise

# 2. 이미지 수집
IMAGE_EXTENSIONS = ("*.jpeg", "*.jpg", "*.png", "*.bmp", "*.webp")
image_files = []

for ext in IMAGE_EXTENSIONS:
    image_files.extend(glob(os.path.join(LOCAL_FOLDER, ext)))

print(f"총 {len(image_files)}개 이미지 파일을 찾았습니다.")

# 3. 이미지 업로드 및 URL 저장
url_list = [] # 결과 저장할 리스트

for file_path in image_files:
    file_name = os.path.basename(file_path)

    try:
        # 업로드
        s3.upload_file(file_path, BUCKET_NAME, file_name)

        # MIME 타입 추정
        mime_type, _ = mimetypes.guess_type(file_name)
        if not mime_type:
            mime_type = "application/octet-stream"

        # Signed URL 생성
        signed_url = s3.generate_presigned_url(
            "get_object",
            Params={
                "Bucket": BUCKET_NAME,
                "Key": file_name,
                "ResponseContentDisposition": "inline",
                "ResponseContentType": mime_type
            },
            ExpiresIn=3600
        )

        print(f"URL: {signed_url}")
        url_list.append(signed_url)

    except ClientError as e:
        print(f"업로드 실패: {e}")


print("모든 이미지 업로드 및 링크 생성 완료!")


### Convert

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_naver import ChatClovaX

chat_llm = ChatClovaX(
    model="HCX-005"
)

# 이미지 URL
image_url = url_list[-1]

# System, User prompt 구성
system_message = SystemMessage(
    content=(
        "당신은 문서 내 다양한 형태의 이미지를 분석하여, 검색 기반 질문응답 시스템(RAG)에 활용 가능한 텍스트 설명을 생성하는 AI입니다."
        "이미지는 인포그래픽, 표, 그래프, 코드 캡처, 다이어그램, 화면 구성 등 다양한 유형일 수 있으며, 다음 기준에 따라 요약을 작성하세요."
        "- 이미지의 주제와 목적을 명확하게 파악하고 자연어로 요약합니다."
        "- 이미지가 전달하는 구조나 흐름이 있다면 순차적으로 설명합니다. (예: 단계, 관계, 비교 등)"
        "- 표, 그래프, 수치 정보는 전체 흐름과 특징적인 차이만 요약하고, 수치 나열은 피합니다."
        "- 코드 캡처인 경우 기능과 역할 중심으로 요약하며, 함수/변수/모듈명 등 핵심 정보만 포함합니다."
        "- 시각적 요소(색상, 도형, 배치 등)는 정보 전달에 필요할 경우에만 간단히 설명합니다."
        "- OCR로 추출된 텍스트가 있다면 핵심 내용 위주로 정리하여 포함합니다."
        "- 설명은 검색 가능한 핵심 키워드를 포함하고, 감상이나 해석 없이 사실 중심 문장으로 구성해야 합니다."
        "- 최종 출력은 3~5문장 이내의 단일 문단으로 구성되며, RAG 시스템의 컨텍스트로 직접 활용 가능해야 합니다."
    )
)
human_message = HumanMessage(content=[
        {"type": "text", "text": "이 이미지는 문서 내 시각 자료입니다. 핵심 정보를 요약해 주세요."},
        {"type": "image_url", "image_url": {"url": image_url}}
    ])

# 메시지 구성
messages = [
    system_message,
    human_message
    ]

# 파라미터 설정
config={
        "generation_config": {
            "temperature": 0.25,
            "repetition_penalty": 1.1
        }
    }

# 모델 호출
response = chat_llm.invoke(messages,config)
print("[CLOVA 응답]\n", response.content)


In [ ]:
# 결과를 저장할 딕셔너리
image_summary_results = []

# URL 반복 → 프롬프트 생성 → 모델 호출 → 딕셔너리 저장
for url in url_list:
    file_name = os.path.basename(url)
    clean_filename = file_name.split("?")[0]
    try:
        # URL만 바꿔서 human_message 재생성
        human_message.content[1]["image_url"]["url"] = url
        messages = [system_message, human_message]
        response = chat_llm.invoke(messages,config)

        # 결과 딕셔너리에 저장
        image_summary_results.append({clean_filename: response.content})
        print(f"[✔] 저장 완료: {url}")

    except Exception as e:
        print(f"[✘] 실패: {url} → {e}")


이미지를 document 변환

In [ ]:
image_docs = []
for item in image_summary_results:
    # 각 딕셔너리에서 파일명과 요약 텍스트 추출
    file_name = list(item.keys())[0]
    summary = item[file_name]

    # 정규식으로 페이지 번호 추출
    match = re.search(r'page_(\d+)_img_\d+\.\w+', file_name)
    page_number = int(match.group(1)) if match else None

    # LangChain Document 생성
    image_docs.append(Document(
        page_content=summary,
        metadata={
            "source": "modeltuning.pdf",
            "page": page_number,
            "images": file_name
        }
    ))

print(f"총 {len(image_docs)}개의 Document 생성 완료")
print(image_docs[0].page_content)
print(image_docs[0].metadata)  # 하나 확인

### Chunking

In [ ]:
# -*- coding: utf-8 -*-

class CompletionExecutor:
    def __init__(self, host, api_key, request_id):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'Authorization': self._api_key,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/segmentation', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['topicSeg']
        else:
            print("[CLOVA 응답 오류]", res['status'])
            return 'Error'
        
file_path = "data/extracted_images_문서/merged_text.txt"

with open(file_path, "r", encoding="utf-8") as f:
    text_content = f.read()

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='clovastudio.stream.ntruss.com',
        api_key="Bearer "+os.environ["CLOVASTUDIO_API_KEY"], # 여기 키 형식이 Bearer이 붙네요 
        request_id=str(uuid.uuid4())
    )

    chunked_docs = []

    for doc in docs:  # docs는 페이지별로 추출한 Document 리스트
        segments = completion_executor.execute(
            # 이전 블로그 참고해 파라미터 설정
            {"postProcessMaxSize": 100,   # 후처리 시 하나의 문단이 가질 수 있는 최대 글자 수 (예: 1000자 이하로 잘라줌)
            "alpha": -100,                # 문단 나누기 민감도 조절 파라미터 (기본: 0.0 / -100으로 두면 자동 조정) - 값이 클수록 더 잘게 나뉘고, 작을수록 덜 나뉨
            "segCnt": -1,                 # 원하는 문단 개수 설정 (-1이면 자동 분할, 1 이상의 정수 입력 시 해당 개수로 고정)
            "postProcessMinSize": -1,     # 후처리 시 하나의 문단이 가져야 할 최소 글자 수 (예: 300자 이상 유지)
            "text": doc.page_content,     # 실제 분할할 원본 텍스트
            "postProcess": True}          # 후처리 여부 설정 (True: 문단 길이 균일화 / False: 모델 출력 그대로 사용)
        )

    for seg in segments:
        chunked_docs.append(Document(
            page_content=' '.join(seg),
            metadata=doc.metadata
        ))    

    print(chunked_docs)
    print("chunk 개수 :",len(chunked_docs))

In [ ]:
# image_docs를 chunked_docs에 추가 (원본은 그대로 유지)
combined_docs = chunked_docs + image_docs

print(f"전체 chunk 개수: {len(combined_docs)}")
print(combined_docs)

In [ ]:
# 샘플 청크 출력
print("\n샘플 청크 (처음 3개):")
for i, chunk in enumerate(combined_docs[:3], 0):
    print(f"\n청크 {i+1}:")
    print(f"내용: {chunk.page_content}")
    print(f"metadata: {chunk.metadata}")
    print(f"길이: {len(chunk.page_content)}자")

### Embedding

In [ ]:
from langchain_naver import ClovaXEmbeddings
 
clovax_embeddings = ClovaXEmbeddings(model='bge-m3') # 임베딩 모델을 설정

text = "임베딩 사용 예제입니다~"
 
clovax_embeddings.embed_query(text)

### Vector Store

In [ ]:
#chroma 다운받기
%pip install -qU langchain-chroma

In [ ]:
import chromadb
from langchain_chroma import Chroma


# 임베딩 모델 정의
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')

# 로컬 클라이언트 생성
client = chromadb.PersistentClient(path="./Chroma_langchain_db123")

# 컬렉션 준비 (이름 중복 주의!)
collection_name = "clovastudiodatas_docs"
client.get_or_create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}
)

# 벡터스토어 객체 생성
vectorstore_Chroma = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=clovax_embeddings
)

# 문서 추가: 최신 방식은 vectorstore.add_documents 사용
print("Adding documents to Chroma vectorstore...")
for doc in combined_docs:
    try:
        vectorstore_Chroma.add_documents([doc])
        time.sleep(1.1) 
    except Exception as e:
        print(f"[✘] 실패: {doc.metadata} → {e}")

print("All documents have been added to the vectorstore.")


In [ ]:
#FAISS 다운로드
%pip install -qU langchain-community faiss-cpu

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# 임베딩 모델 정의
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')

# FAISS 인덱스 생성 (1024는 bge-m3 차원 수에 맞춰야 함)
index = faiss.IndexFlatIP(1024)  # 내적 기반 검색

# FAISS 벡터스토어 생성
vectorstore_FAISS = FAISS(
    embedding_function=clovax_embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# 문서 일괄 추가 (자동 임베딩 처리)
print("Adding documents to FAISS vectorstore...")
for doc in combined_docs:
    try:
        vectorstore_FAISS.add_documents([doc])
        time.sleep(1.1) 
    except Exception as e:
        print(f"[✘] 실패: {doc.metadata} → {e}")
print("All documents have been added to FAISS vectorstore.")


## 3. 질의하기

### 질문하기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA

# System 및 User 메시지를 나눠 구성
system_template = (
    "당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 원래 가지고있는 지식은 모두 배제하고, 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다."
    "만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요."
)
user_template = (
    "다음은 검색된 문서 내용입니다:\n\n{context}\n\n"
    "위 정보를 바탕으로 다음 질문에 답해주세요:\n{question}"
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template),
])

# 원하는 vectorstore 선택해서 사용
retriever = vectorstore_Chroma.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1, "k": 3}
    )
# retriever = vectorstore_FAISS.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={"score_threshold": 0.1, "k": 3}
# )

# Retrieval QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True
)

# 실행
question = "데이터셋 규모가 커질수록 2대륙의 오류 발생 확률은 어떻게 돼?"
result = qa_chain.invoke({"query": question})

print("질문:", question)
print("응답:", result["result"])  # 모델의 실제 응답
for i, doc in enumerate(result["source_documents"]): # 답변시 참고 한 문서
    print(f"\n[출처 문서 {i+1}]\n내용: {doc.page_content}\n메타데이터: {doc.metadata}")